<a href="https://www.kaggle.com/code/datascientistsalman/unlocking-magnus-carelson-chess-playing-strategies?scriptVersionId=210686278" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**Created by:** Salman Ahmed  
**LinkedIn:** [Salman Ahmed ](https://www.linkedin.com/in/salmanahmed84/)  
**Portfolio:** [Salman's Portfolio](https://github.com/salman-datascientist)

**Gemini 1.5 with Long-Context Windows for Analyzing Hours-Long Chess Game Videos**

***Extended Context Length:***
Vast Input Capacity: Gemini 1.5 Pro excels at processing hours-long chess game videos, managing inputs of up to 10 million tokens. This allows for seamless analysis of entire tournaments without segmenting the data into smaller chunks​.
Improved Contextual Understanding: The extended context length enables the model to connect strategic decisions across games, identify long-term patterns, and correlate player behavior over time.

***High Recall:***
Accurate Information Retrieval: When analyzing hours of video, Gemini 1.5 Pro can precisely locate specific moves, strategies, or key moments, such as a player executing a checkmate or pivotal errors in gameplay. This high recall ensures no critical detail is missed​.
Superior Performance: Compared to other models, Gemini 1.5 Pro demonstrates unmatched accuracy in retrieving and reasoning over long video sequences, even outperforming specialized models like GPT-4V for video QA.
Multimodal Capabilities:

***Unified Understanding:***
Gemini 1.5 Pro integrates video, audio commentary, and text-based annotations in chess games to create a comprehensive analysis of gameplay strategies and decisions​(gemini_v1_5_report).
Enhanced Contextual Awareness: By combining visual inputs (e.g., board state), audio (e.g., player commentary), and text (e.g., move notations), the model delivers richer insights into player strategies and decision-making processes.
Efficiency and Scalability:

***Optimized Processing:*** 
Designed to handle extensive data, Gemini 1.5 Pro efficiently processes multiple hours of chess gameplay videos, ensuring that even tournaments spanning several days can be analyzed in one session.
Scalability: Its ability to analyze entire archives of chess games, such as championship series, makes it ideal for researchers, commentators, or AI systems studying gameplay evolution and strategy trends​.
Reduced Reliance on External Tools:

***Self-Sufficiency:***
Unlike systems that rely on external video processing or annotation tools, Gemini 1.5 Pro can directly analyze video content, extracting timestamps, moves, and strategic insights without additional preprocessing​.

***Simplified Workflow:***
This all-in-one capability simplifies the workflow, enabling faster and more efficient analysis of chess games without requiring integration with separate retrieval or indexing systems​.

Install the SDK

In [ ]:
%pip install -U -q "google-generativeai>=0.8.3"

<center>
  <h2 style="color: blue;">YouTube Video: <a href="https://www.youtube.com/watch?v=RkBgf06G89E" target="_blank" style="color: blue;">Magnus Carlsen goes 94% BERSERK in Blitz Titled Arena</a></h2>
</center>

In [ ]:
from IPython.display import HTML

# Display a centered YouTube video
video_id = "RkBgf06G89E"
HTML(f"""
<div style="display: flex; justify-content: center;">
    <iframe width="1024" height="576" src="https://www.youtube.com/embed/{video_id}" frameborder="0" allowfullscreen></iframe>
</div>
""")

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemini-long-context/submission_instructions.txt
/kaggle/input/magnus-chess-blitz/magnus_chess_blitz.mp4


In [9]:
import os
import time
import datetime
import google.generativeai as genai
from google.generativeai import caching
from IPython.display import HTML, Markdown, display, Video

Get GEMINI_API_KEY stored in User Secrets.

In [3]:
# Get the API key from here: https://ai.google.dev/tutorials/setup
# Create a new secret called "GEMINI_API_KEY" via Add-ons -> Secrets in the top menu, and attach it to this notebook.

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


**Choice of Gemini Model:**

gemini-1.5-flash, alias that points to gemini-1.5-flash-002
Best for:
Image understandin, 
Video understandi, g
Audio understanding
Use case:
Process 3,000 images at a, time
Look through 1 hour long , ideos
Listen to hours o
f audio
!Testing ! - Run your first prompt
In this step, you will test that your API key is set up correctly by making a request. The gemini-1.5-flash model has been selected here.

In [16]:
# found 'gemini-1.5-flash-001' response to be vastly different and more useful, but still went with latest version of gemini flash
#flash = genai.GenerativeModel('gemini-1.5-flash-001')
flash = genai.GenerativeModel('gemini-1.5-flash-002')
response = flash.generate_content("Explain AI to me like I'm a kid.")
print(response.text)

Imagine you have a super smart friend who's really good at learning. This friend is called AI, short for Artificial Intelligence.

AI is like a computer that can think and learn like a human. It can learn from all the information you give it, like books, pictures, or even your own words!

Think about how you learn to play a game. First, you see how it's done, then you practice, and you get better over time. AI works the same way! It looks at lots of examples and then uses what it learned to solve problems or do tasks.

Here are some things AI can do:

* **Play games:** AI can play games like chess or video games better than humans sometimes!
* **Help you find things:** AI helps you find what you're looking for online, like when you search for a video on YouTube.
* **Translate languages:** AI can help you understand what people are saying in other languages.
* **Drive cars:** AI can help drive cars safely and without human drivers.

It's like having a super smart friend that can help yo

***When to use context caching***

Context caching is particularly well suited to scenarios where a substantial initial context is referenced repeatedly by shorter requests. Consider using context caching for use cases such as:
* Chatbots with extensive system instructions
* Repetitive analysis of lengthy video files
* Recurring queries against large document sets
* Frequent code repository analysis or bug fixingtput tokens.

***How caching reduces costs***

Context caching is a paid feature designed to reduce overall operational costs. Billing is based on the following factors:

1. **Cache token count**: The number of input tokens cached, billed at a reduced rate when included in subsequent prompts.
2. **Storage duration**: The amount of time cached tokens are stored (TTL), billed based on the TTL duration of cached token count. There are no minimum or maximum bounds on the TTL.
3. **Other factors**: Other charges apply, such as for non-cached input tokens and output tokens.


In [ ]:
magnus_video1 = '/kaggle/input/magnus-chess-blitz/magnus_chess_blitz.mp4'
Video(magnus_video1, embed=True, width=640, height=480)

In [ ]:
# Upload the video using the Files API
video_file = genai.upload_file(path=magnus_video1)

# Wait for the file to finish processing
while video_file.state.name == 'PROCESSING':
  print('Waiting for video to be processed.')
  time.sleep(2)
  video_file = genai.get_file(video_file.name)

print(f'Video processing complete: {video_file.uri}')

# Create a cache with a 5 minute TTL
cache = caching.CachedContent.create(
    model='models/gemini-1.5-flash-002',
    display_name='magnus chess blitz of 51 games', # used to identify the cache
    system_instruction=(
        'You are an expert video analyzer, and your job is to answer '
        'the user\'s query based on the video file you have access to.'
    ),
    contents=[video_file],
    ttl=datetime.timedelta(minutes=5),
)

# Construct a GenerativeModel which uses the created cache.
model = genai.GenerativeModel.from_cached_content(cached_content=cache)

In [14]:
import json
import glob

# Directory containing your JSON files
data_folder = "/kaggle/input/magnus-chess-blitz"
# List all JSON files in the directory
json_files = glob.glob(os.path.join(data_folder, "*.json"))


data_list = []
for json_file in json_files:
    # Read the content of the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    # Convert JSON data to a string
    data_str = json.dumps(data)
    data_list.append(data_str)
    print(json_file, flash.count_tokens(data_str))

print('Token size of combined video files', flash.count_tokens(data_list))

NameError: name 'gemini_model' is not defined

Define methods to upload large data/video files.

In [13]:
def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()

In [ ]:
# delete file
genai.delete_file(files[0].name)

## Bibliography
* https://aistudio.google.com/app/apikey
* https://storage.googleapis.com/deepmind-media/gemini/gemini_v1_5_report.pdf
* https://ai.google.dev/gemini-api/docs/text-generation?lang=python
* https://ai.google.dev/api/generate-content#image